<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/Valuation_Arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [3]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [4]:
from ast import increment_lineno
# Extrair retornos ajustados da empresa e do benchmark

retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]



[*********************100%***********************]  2 of 2 completed


In [5]:
# Começando a regressão

x = sm.add_constant(x)

modelo = sm.OLS(y, x)

resultados = modelo.fit()

print(resultados.summary())
# Extraindo valor do Beta

beta = resultados.params[1]

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]
upper_bound = conf_interval.iloc[1, 1]

print("Intervalo de Confiança:")
print("Limite Inferior:", lower_bound)
print("Limite Superior:", upper_bound)


                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Sun, 18 Jun 2023   Prob (F-statistic):           5.30e-09
Time:                        03:19:32   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [6]:
# Cálculo do Custo do equity

# Desalavancando beta

tax = 0.27
debt = 1060093
equity = 2716661

unlev_beta = beta/(1+(1-tax)*debt/equity)

# Risk Free Obtido 14/06
rf = 0.1096

# Default spread 14/06
dfs = 0.0314



# Calculando equity premium

data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download(indice, start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = retornos.dropna()

retorno_anualizado = (float(retornos.mean())+1)**12-1
print(retorno_anualizado)

[*********************100%***********************]  1 of 1 completed
0.08755353683124478


In [7]:
path = "https://github.com/Bmiguez/Arezzo-Valuation/raw/main/Dados_Valuation.xlsx"
aba_bp = "BP"
aba_dre = "DRE"
aba_fc = "FC"
aba_lp = "LojasProp"
aba_indicadores = "Indicadores"
aba_receitabruta = "Receita Bruta"

#### Criação dos Dataframes
#### Balanço Patrimonial
df_bp = pd.read_excel(path, aba_bp)

#### Demonstração Resultado do Exercício
df_dre = pd.read_excel(path, aba_dre)

#### Fluxo de Caixa
df_fc = pd.read_excel(path, aba_fc)

#### Lojas Próprias
df_lp = pd.read_excel(path, aba_lp)

#### Indicadores
df_ind = pd.read_excel(path, aba_indicadores)

#### Receita Bruta
df_receita = pd.read_excel(path, aba_receitabruta)

In [9]:
# Calculando a média das deduções entre receita bruta e liquida
receita_bruta = df_receita.iloc[:,1]
receita_liquida = df_dre.iloc[:,1]

deducoes = pd.DataFrame({"Ano": range(2010,2023), "Percentual":receita_liquida/receita_bruta})

## Calculando a média das deduções nos últimos cinco anos
deducoes["Média_5_anos"] = deducoes["Percentual"].rolling(5).mean().shift()



### Formulando premissas de receita com base no numero de lojas

df_lojas_prop = df_lp.iloc[:,[0] + list(range(15,25))].apply(pd.to_numeric, errors='coerce')

franquias = df_lp.iloc[:,[0] + list(range(6,14))].apply(pd.to_numeric, errors='coerce')

## Cálculo do número de aberturas de lojas prórprias por marca
aberturas_LP = df_lojas_prop.iloc[:,1:].diff()
aberturas_LP = aberturas_LP.reindex(columns=["Ano"] + aberturas_LP.columns[:-1].tolist())
aberturas_LP["Ano"] = df_lojas_prop.iloc[:, 0]
aberturas_LP = aberturas_LP.drop(aberturas_LP.index[0])

# Cálculo da média dos últimos 3 anos destas aberturas

df_mediaslp_aberturas = aberturas_LP.iloc[:,1:].rolling(3).mean().shift()
df_mediaslp_aberturas = df_mediaslp_aberturas.reindex(columns=["Ano"] + df_mediaslp_aberturas.columns[:-1].tolist())
df_mediaslp_aberturas["Ano"] = df_lojas_prop.iloc[:, 0]


## Cálculo do número de aberturas de franquias por marca

aberturas_FR = franquias.iloc[:,1:].diff()
aberturas_FR = aberturas_FR.reindex(columns=["Ano"] + aberturas_FR.columns[:-1].tolist())
aberturas_FR["Ano"] = franquias.iloc[:, 0]
aberturas_FR = aberturas_FR.drop(aberturas_FR.index[0])

# Cálculo da média dos últimos 3 anos destas aberturas

df_mediasfr_aberturas = aberturas_FR.iloc[:,1:].rolling(3).mean().shift()
df_mediasfr_aberturas = df_mediasfr_aberturas.reindex(columns=["Ano"] + df_mediasfr_aberturas.columns[:-1].tolist())
df_mediasfr_aberturas["Ano"] = df_lojas_prop.iloc[:, 0]


# Cálculo do mix de receita por canal por ano
canais = ["Franquias", "Multimarcas", "Lojas próprias", "Web Commerce", "Outros²"]

df_canais = df_receita.loc[:,canais]
df_particip_canais = df_canais.apply(lambda row: row / row.sum(), axis=1)

### Cálculo da receita por loja/receita por franquia
## Obter receita de lojas e receita de franquias de cada marca

#Arezzo

receitalp_arezzo = df_receita.loc[:,"Arezzo"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_arezzo = df_receita.loc[:,"Arezzo"] * df_particip_canais.loc[:,"Franquias"]

#Schutz

receitalp_schutz = df_receita.loc[:,"Schutz"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_schutz = df_receita.loc[:,"Schutz"] * df_particip_canais.loc[:,"Franquias"]

#Anacapri
receitalp_anacapri = df_receita.loc[:,"Anacapri"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_anacapri = df_receita.loc[:,"Anacapri"] * df_particip_canais.loc[:,"Franquias"]

#AR&CO
receitalp_arco = df_receita.loc[:,"AR&CO"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_arco = df_receita.loc[:,"AR&CO"] * df_particip_canais.loc[:,"Franquias"]

#Outros
receitalp_outros = df_receita.loc[:,"Outros¹"] * df_particip_canais.loc[:,"Lojas próprias"]
receitafr_outros = df_receita.loc[:,"Outros¹"] * df_particip_canais.loc[:,"Franquias"]

## Receita por lojas próprias
df_receitalp_total = df_lojas_prop.iloc[1:,:]
print(df_receitalp_total)

rec_porLP_arr= receitalp_arezzo / df_receitalp_total["Arezzo.1"].reset_index(drop=True)
rec_porFR_arr= receitafr_arezzo / franquias.loc[1:,"Arezzo"].reset_index(drop=True)

rec_porLP_sch= receitalp_schutz / df_receitalp_total["Schutz.1"].reset_index(drop=True)
rec_porFR_sch= receitafr_schutz / franquias.loc[1:,"Schutz"].reset_index(drop=True)

rec_porLP_arc= receitalp_arco / df_receitalp_total["AR&CO.1"].reset_index(drop=True)
rec_porFR_arc= receitafr_arco / franquias.loc[1:,"AR&CO"].reset_index(drop=True)

rec_porLP_ana= receitalp_anacapri / df_receitalp_total["Anacapri.1"].reset_index(drop=True)
rec_porFR_ana= receitafr_anacapri / franquias.loc[1:,"Anacapri"].reset_index(drop=True)

rec_porLP_out= receitalp_outros / df_receitalp_total["Anacapri.1"].reset_index(drop=True)
rec_porFR_out= receitafr_outros / franquias.loc[1:,"Anacapri"].reset_index(drop=True)

     Ano  Arezzo.1  Schutz.1  Alexandre Birman  Anacapri.1  Fiever.1  Alme.1  \
1   2010        13        10                 1           5       NaN     NaN   
2   2011        19        17                 1           8       NaN     NaN   
3   2012        19        27                 2           8       NaN     NaN   
4   2013        17        27                 2           8       NaN     NaN   
5   2014        19        27                 2           5       NaN     NaN   
6   2015        15        26                 2           4       1.0     NaN   
7   2016        15        23                 3           4       3.0     NaN   
8   2017        15        22                 4           3       4.0     NaN   
9   2018        14        17                 4           3       5.0     2.0   
10  2019        10        17                 6           3       5.0     3.0   
11  2020        12        16                 8           5       2.0     2.0   
12  2021        18        20            

1º Valuation

2015-2022